# Part1

## Q1

Both functions exploit the identity: ln(x) = 2^n ln(x^1/2^n). when the argument is very close to 1, ln(1+u) ~= u. So they repeatedly take square roots to push the argument toward 1, then multiply the tiny log back by 2^n.

**`naivelog`** : After n square roots, y= x^1/2^n. Then approximate ln(y) by y-1 (i.e. ln(1+u) ~= u = y-1). Then scale it back and return 2^n(y-1). Since y->1 as n gros, the truncation error behaves like lnx - 2^n(y-1) = 2^n[ln(1+(y-1))-(y-1)] = -2^n((y-1)^2)/+ O(2^n(y-1)^3). The truncation terms shrink roughly like O[(lnx)^2)/2^n]. 

Roundoff: For large n, y is extremely close to 1 and the subtraction y-1 suffers catastrophic cancellation in floating-point. You accumulated roundoff through n square roots, thus there's a sweet spot for n. When 0<x<1, square roots push y = x^1/2^n up toward 1, and lnx <0 falls out naturally.

**`mylog`**: z<- z/ (1 + sqrt(1+z)), starting wwith z0 = x-1. It avoids substracting nearly equal numbers by computing sqrt(1+z) - 1 via the division to reduce cancellation. Then 1=Zk+1 = sqrt(1+Zk) --> 1 + Zn =  (1+Z0)^1/2^n = x^1/2^n. After n steps, Zn = x^1/2^n -1, then retrun 2^n Zn ~= lnx

As n increases, truncation error gets smaller, and more round off accumulation. For naivelog function, the final y-1 substraction loses digits badly. MYLOG function keeps the same asymptotic truncation but greatly reduces cancellation, so it typically wins for larger n or when x is close to 1.

## Q2

In [44]:

"""


  naivelog(x):
      y := x; repeat n times: y := sqrt(y)          # y = x^(1/2^n)
      return 2^n * (y - 1)                          # ln y ≈ y - 1 when y≈1

  mylog(x):
      z := x - 1
      repeat n times: z := z / (1 + sqrt(1 + z))    # stable form of (sqrt(1+z) - 1)
      return 2^n * z

Key identity both use:
    ln x = 2^n * ln( x^(1/2^n) ), and ln(1+u) ≈ u for small u.


  • For integer x ∈ {1,…,14} and moderate n (15–35), both methods give
    essentially the same numbers in double precision (mean/max |err| match).
    The reason is that x is not super close to 1, and n in this range keeps truncation
    small without making (y-1) so tiny that cancellation dominates.
  • For fractional x in (0,1), especially near 1 and at larger n (e.g., n=35),
    naivelog begins to lose digits, while mylog remains near machine precision (~1e-11 to 1e-12).
    The reason is that naivelog’s final subtraction y-1 suffers catastrophic cancellation
    when y ≈ 1 after many square-roots; mylog’s update avoids the subtraction.

"""

import math
from typing import Iterable, List, Tuple, Dict


def naivelog(x: float, n: int = 15) -> float:
    """
    Argument-halving log (naive subtraction form).
    y := x; repeat n times: y := sqrt(y)     ->  y = x^(1/2^n)
    ln(x) ≈ 2^n * (y - 1)

    Caveat: For large n, y ≈ 1 and (y - 1) loses many digits (cancellation).
    """
    y = float(x)
    for _ in range(n):
        y = math.sqrt(y)
    l = y - 1.0
    for _ in range(n):
        l *= 2.0
    return l


def mylog(x: float, n: int = 15) -> float:
    """
    Argument-halving log (cancellation-avoiding form).
    z_0 = x - 1; iterate
        z_{k+1} = sqrt(1 + z_k) - 1  =  z_k / (1 + sqrt(1 + z_k))   (stable form)
    After n steps: 1 + z_n = x^(1/2^n), so ln x ≈ 2^n * z_n.
    """
    z = float(x) - 1.0
    for _ in range(n):
        z = z / (1.0 + math.sqrt(1.0 + z))
    l = z
    for _ in range(n):
        l *= 2.0
    return l


# ---------------------------------
# Evaluation / reporting utilities
# ---------------------------------

def _safe_rel_err(approx: float, true: float) -> float:
    """Relative error with a small safeguard for true≈0."""
    denom = max(1.0, abs(true))
    return abs(approx - true) / denom

def _summaries_over_x(n_values: Iterable[int],
                      xs: Iterable[float]) -> List[Dict[str, float]]:
    """
    For each n in n_values, compute mean and max |error| across xs
    for both naivelog and mylog.
    """
    rows: List[Dict[str, float]] = []
    for n in n_values:
        errs_naive: List[float] = []
        errs_my:    List[float] = []
        for x in xs:
            true = math.log(x)
            naive = naivelog(x, n=n)
            my    = mylog(x, n=n)
            errs_naive.append(abs(naive - true))
            errs_my.append(abs(my - true))
        rows.append({
            "n": n,
            "mean_abs_err_naivelog": sum(errs_naive)/len(errs_naive),
            "max_abs_err_naivelog":  max(errs_naive),
            "mean_abs_err_mylog":    sum(errs_my)/len(errs_my),
            "max_abs_err_mylog":     max(errs_my),
        })
    return rows


def _print_summary_table(rows: List[Dict[str, float]], title: str) -> None:
    print("\n" + title)
    print("-" * len(title))
    header = (" n  |   mean|err| naive    max|err| naive      "
              "mean|err| mylog     max|err| mylog")
    print(header)
    print("-" * len(header))
    for r in rows:
        print(f"{r['n']:>3d} | "
              f"{r['mean_abs_err_naivelog']:.3e}    {r['max_abs_err_naivelog']:.3e}     "
              f"{r['mean_abs_err_mylog']:.3e}    {r['max_abs_err_mylog']:.3e}")


def _print_per_x_table(n: int, xs: Iterable[float], title: str) -> None:
    print("\n" + title + f" (n = {n})")
    print("-" * (len(title) + 8))
    print("   x     true ln(x)        naivelog           |err|       rel.err     "
          "      mylog             |err|       rel.err")
    print("---------------------------------------------------------------------------------------------------------")
    for x in xs:
        true  = math.log(x)
        naive = naivelog(x, n=n)
        my    = mylog(x, n=n)
        err_n = abs(naive - true)
        err_m = abs(my - true)
        r_n   = _safe_rel_err(naive, true)
        r_m   = _safe_rel_err(my, true)
        print(f"{x:6.2f}  {true: .12e}  {naive: .12e}  {err_n: .3e}  {r_n: .3e}   "
              f"{my: .12e}  {err_m: .3e}  {r_m: .3e}")


def _print_near_one_table(n: int, deltas=(1e-2, 1e-3, 1e-4, 1e-5), title="Near-1 stress test") -> None:
    """
    Probe x extremely close to 1 to expose cancellation in naivelog at large n.
    We test x = 1±δ with progressively smaller δ.
    """
    xs = []
    for d in deltas:
        xs += [1.0 - d, 1.0 + d]
    xs.sort()
    print("\n" + f"{title} (n = {n})")
    print("-" * (len(title) + 12))
    print("      x           true ln(x)         naivelog            |err|         mylog              |err|")
    print("------------------------------------------------------------------------------------------------")
    for x in xs:
        true  = math.log(x)
        nv    = naivelog(x, n=n)
        mv    = mylog(x, n=n)
        print(f"{x: .10f}  {true: .12e}  {nv: .12e}  {abs(nv-true): .3e}   {mv: .12e}  {abs(mv-true): .3e}")


# --------------
# RUN
# --------------

if __name__ == "__main__":
    # Range from the lab: integers 1..14 
    xs_int = list(range(1, 15))

    # Also probe (0,1), where ln(x) < 0. 
    xs_frac = [0.1, 0.2, 0.5, 0.9]

    # Try several n to show truncation vs roundoff trade-off.
    n_values = [15, 20, 25, 30, 35]

    # (A) Summaries over x = 1..14 
    rows_int = _summaries_over_x(n_values, xs_int)
    _print_summary_table(rows_int, "Summary over x = 1..14")

    # (B) Detailed per-x table for n = 15 
    _print_per_x_table(15, xs_int, "Per-x errors (x = 1..14)")

    # (C) Behavior in (0,1): try n = 15 and n = 35 to expose cancellation in naivelog
    _print_per_x_table(15, xs_frac, "Per-x errors for fractional x (0<x<1)")
    _print_per_x_table(35, xs_frac, "Per-x errors for fractional x (0<x<1)")

    # (D) EXTRA: Direct “near-1” stress test (highly illustrative at larger n)
    _print_near_one_table(35, deltas=(1e-2, 1e-3, 1e-4, 1e-5))




Summary over x = 1..14
----------------------
 n  |   mean|err| naive    max|err| naive      mean|err| mylog     max|err| mylog
---------------------------------------------------------------------------------
 15 | 5.790e-05    1.063e-04     5.790e-05    1.063e-04
 20 | 1.809e-06    3.321e-06     1.809e-06    3.321e-06
 25 | 5.658e-08    1.055e-07     5.654e-08    1.038e-07
 30 | 1.064e-07    2.186e-07     1.767e-09    3.243e-09
 35 | 3.130e-06    7.089e-06     5.521e-11    1.014e-10

Per-x errors (x = 1..14) (n = 15)
--------------------------------
   x     true ln(x)        naivelog           |err|       rel.err           mylog             |err|       rel.err
---------------------------------------------------------------------------------------------------------
  1.00   0.000000000000e+00   0.000000000000e+00   0.000e+00   0.000e+00    0.000000000000e+00   0.000e+00   0.000e+00
  2.00   6.931471805599e-01   6.931545117404e-01   7.331e-06   7.331e-06    6.931545117428e-01   7.331

The functions **`naivelog`** and **`mylog`** behave differently because of how they handle numerical stability when (x) is close to 1. Both rely on the identity lnx = 2^n(ln(x^{1/2^n}) ~= 2^n(x^{1/2^n} -1 ) but `naivelog` computes (x^{1/2^n} - 1) directly after many square-root iterations. When (n) is large, (x^{1/2^n}) becomes extremely close to 1, so subtracting two nearly equal floating-point numbers causes **catastrophic cancellation**, losing many significant digits. In contrast, `mylog` uses the algebraically equivalent but numerically stable recurrence (z_{k+1} = z_k / (1 + sqrt{1 + z_k})), which avoids the subtraction of nearly equal quantities. Quantitatively, for moderate (x) (e.g., 1–14) and (n ≤ 20), both achieve machine-level precision with absolute errors around (10^{-6})–(10^{-8}). However, for (x) near 1 and large (n) (e.g., (n=35)), `naivelog`’s error grows to about (10^{-6})–(10^{-7}) due to roundoff amplification, while `mylog` maintains stability with errors near (10^{-11})–(10^{-12}). Thus, `mylog`’s reformulation preserves precision at high iteration counts by eliminating the main source of floating-point cancellation.


## Q3

In [45]:
# Functions:
#   exp_halving(x, n_halve=8, tol=1e-16)
#   sin_halving(x, n_halve=6, tol=1e-16)
#   cos_halving(x, n_halve=6, tol=1e-16)
#   atan_halving(u, n=20)
#
# Demos are at the bottom (π via 4*atan(1), and accuracy checks).

import math
from math import sqrt

# ---------- Helpers: tiny-argument series ----------
def _taylor_exp_small(y, tol=1e-16, max_terms=100):
    term = 1.0
    s = 1.0
    for k in range(1, max_terms+1):
        term *= y / k
        s += term
        if abs(term) < tol * max(1.0, abs(s)):
            break
    return s

def _taylor_sin_cos_small(y, tol=1e-16, max_terms=80):
    s = y
    c = 1.0
    term_s = y
    term_c = 1.0
    k = 1
    while k < max_terms:
        term_c *= -y*y / ((2*k-1)*(2*k))
        c += term_c
        term_s *= -y*y / ((2*k)*(2*k+1))
        s += term_s
        if max(abs(term_s), abs(term_c)) < tol * max(1.0, abs(c), abs(s)):
            break
        k += 1
    return s, c

def _reduce_to_minus_pi_pi(x):
    twopi = 2.0 * math.pi
    x_mod = math.fmod(x, twopi)
    if x_mod <= -math.pi:
        x_mod += twopi
    elif x_mod > math.pi:
        x_mod -= twopi
    return x_mod

# ---------- Q3: exp, sin, cos via argument halving ----------
def exp_halving(x, n_halve=8, tol=1e-16):
    """
    e^x by argument halving:
      1) y = x / 2^n
      2) v ≈ e^y via short Taylor
      3) square back n times: (e^y)^(2^n)
    """
    y = x / (2.0**n_halve)
    v = _taylor_exp_small(y, tol=tol)
    for _ in range(n_halve):
        v *= v
    return v

def sin_cos_halving(x, n_halve=6, tol=1e-16):
    """
    (sin x, cos x) by halving + double-angle:
      * reduce x into [-π, π] (periodicity/symmetry)
      * y = x / 2^n; evaluate sin(y), cos(y) by small-angle Taylor
      * double back n times:
          sin(2u) = 2 sin u cos u
          cos(2u) = cos^2 u - sin^2 u
    """
    x_r = _reduce_to_minus_pi_pi(x)
    y = x_r / (2.0**n_halve)
    s, c = _taylor_sin_cos_small(y, tol=tol)
    for _ in range(n_halve):
        s, c = (2.0*s*c, c*c - s*s)
    return s, c

def sin_halving(x, n_halve=6, tol=1e-16):
    s, _ = sin_cos_halving(x, n_halve=n_halve, tol=tol)
    return s

def cos_halving(x, n_halve=6, tol=1e-16):
    _, c = sin_cos_halving(x, n_halve=n_halve, tol=tol)
    return c


## Q4

In [46]:
# ---------- Q4: arctan via argument halving ----------
def atan_halving(u, n=20):
    """
    arctan(u) with the half-angle transform:
      arctan(u) = 2 * arctan( u / (1 + sqrt(1 + u^2)) )
    Iterate:
      z_{k+1} = z_k / (1 + sqrt(1 + z_k^2)), z_0 = u
    For tiny z_n, arctan(z_n) ~ z_n, so return (2^n) * z_n.
    The divided form avoids the catastrophic subtraction in sqrt(1+z) - 1.
    """
    z = float(u)
    for _ in range(n):
        z = z / (1.0 + math.sqrt(1.0 + z*z))
    return (2.0**n) * z

## Demos

In [47]:
# ---------- demos ----------
if __name__ == "__main__":
    # exp demo
    for x in [0.0, 0.5, 1.0, -1.0, 2.0]:
        approx = exp_halving(x, n_halve=8)
        print("exp", x, approx, "err", abs(approx - math.exp(x)))

    # sin/cos demo on 0..1.6 (as the lab suggests)
    xs = [0.1*k for k in range(0, 17)]
    for x in xs:
        s = sin_halving(x, n_halve=6)
        c = cos_halving(x, n_halve=6)
        print("x=", x, "sin_err", abs(s - math.sin(x)), "cos_err", abs(c - math.cos(x)))

    # atan + pi
    for n in (5, 10, 15, 20, 25):
        pi_est = 4.0 * atan_halving(1.0, n=n)
        print("n=", n, "pi_est", pi_est, "abs_err", abs(pi_est - math.pi))

exp 0.0 1.0 err 0.0
exp 0.5 1.6487212707001124 err 1.5765166949677223e-14
exp 1.0 2.7182818284591685 err 1.234568003383174e-13
exp -1.0 0.3678794411714431 err 7.771561172376096e-16
exp 2.0 7.38905609893037 err 2.8066438062523957e-13
x= 0.0 sin_err 0.0 cos_err 0.0
x= 0.1 sin_err 1.1102230246251565e-16 cos_err 1.1102230246251565e-15
x= 0.2 sin_err 4.163336342344337e-16 cos_err 2.1094237467877974e-15
x= 0.30000000000000004 sin_err 1.1657341758564144e-15 cos_err 4.218847493575595e-15
x= 0.4 sin_err 1.609823385706477e-15 cos_err 3.9968028886505635e-15
x= 0.5 sin_err 4.996003610813204e-16 cos_err 9.992007221626409e-16
x= 0.6000000000000001 sin_err 4.6629367034256575e-15 cos_err 7.327471962526033e-15
x= 0.7000000000000001 sin_err 2.55351295663786e-15 cos_err 3.1086244689504383e-15
x= 0.8 sin_err 8.881784197001252e-16 cos_err 9.992007221626409e-16
x= 0.9 sin_err 8.881784197001252e-16 cos_err 7.771561172376096e-16
x= 1.0 sin_err 1.7763568394002505e-15 cos_err 1.2212453270876722e-15
x= 1.1 sin_e

# Part 2

## Q1

The CORDIC algorithm computes tan (θ) using a sequence of simple shift-and-add rotations that successively reduce an initial angle θ to zero. In rotation mode, starting from the vector (1, 0), the algorithm iteratively applies micro-rotations by fixed angles αᵢ = arctan(2⁻ᶦ), updating
(x_{i+1} = x_i - d_i,y_i,2^{-i}), (y_{i+1} = y_i + d_i,x_i,2^{-i}), and (z_{i+1} = z_i - d_i,α_i),
where dᵢ = sign(zᵢ) chooses the rotation direction. After m iterations, the resulting vector (xₘ, yₘ) ≈ (1/Kₘ)(cos θ, sin θ), where K{m} = ∏₀^{m−1}(1 + 2^-2i)^(-1/2) ~= 0.6072529 is a constant scale factor. The tangent is then obtained as tan θ = yₘ / xₘ, with the scale factor canceling out in the ratio. Quantitatively, using m ≈ 30 iterations achieves double-precision accuracy (|error| ~= 10⁻⁹), since each additional step halves the residual angle. Thus, CORDIC replaces multiplications and divisions with fast shifts and adds, making it ideal for hardware or low-cost computation of tan θ.


## Q2

In [48]:


import math
from math import sqrt, pi

# ---------------------------------------------------------------
# Helper 1: arctan via argument-halving (from Part 1)
# ---------------------------------------------------------------
def atan_halving(u: float, n: int = 20) -> float:
    """
    Compute arctan(u) using the stable half-angle recursion.
      z_{k+1} = z_k / (1 + sqrt(1 + z_k^2))
      arctan(u) ≈ (2^n) * z_n
    """
    z = float(u)
    for _ in range(n):
        z = z / (1.0 + math.sqrt(1.0 + z*z))
    return (2.0**n) * z


# ---------------------------------------------------------------
# Helper 2: Precompute αᵢ = atan(2⁻ⁱ) for CORDIC
# ---------------------------------------------------------------
def cordic_angles(m: int = 30, atan_n: int = 25):
    """Return [atan(1), atan(1/2), atan(1/4), ...] using our own atan_halving."""
    return [atan_halving(2.0**(-i), n=atan_n) for i in range(m)]


# ---------------------------------------------------------------
# Helper 3: Reduce any angle θ to [0, π/2] and record sign of sin/cos
# ---------------------------------------------------------------
def reduce_to_first_quadrant(theta: float):
    """
    Reduce θ to [0, π/2] using trig symmetries.
    Returns (reduced_angle, sign_sin, sign_cos).
    """
    twopi = 2.0 * math.pi
    t = math.fmod(theta, twopi)
    if t < 0.0:
        t += twopi
    if t <= 0.5*math.pi:
        a, ss, cc = t, +1.0, +1.0
    elif t <= math.pi:
        a, ss, cc = math.pi - t, +1.0, -1.0
    elif t <= 1.5*math.pi:
        a, ss, cc = t - math.pi, -1.0, -1.0
    else:
        a, ss, cc = 2.0*math.pi - t, -1.0, +1.0
    return a, ss, cc


# ---------------------------------------------------------------
# Helper 4: Core CORDIC iteration — rotation mode
# ---------------------------------------------------------------
def cordic_rotate(theta: float, m: int = 30, angles=None):
    """
    Rotation mode: given θ, iteratively rotate (x,y) to reach that angle.
      x_{i+1} = x_i - d_i * y_i * 2^-i
      y_{i+1} = y_i + d_i * x_i * 2^-i
      z_{i+1} = z_i - d_i * α_i
    After m steps: (x,y) ≈ (1/K_m)(cosθ, sinθ).
    """
    if angles is None:
        angles = cordic_angles(m)
    x, y, z = 1.0, 0.0, theta
    for i in range(m):
        d = 1.0 if z >= 0.0 else -1.0
        x, y, z = x - d*y*(2.0**(-i)), y + d*x*(2.0**(-i)), z - d*angles[i]
    # scale factor
    K = 1.0
    for i in range(m):
        K /= math.sqrt(1.0 + 2.0**(-2*i))
    return x, y, K


# ---------------------------------------------------------------
# Helper 5: Core CORDIC iteration — vectoring mode
# ---------------------------------------------------------------
def cordic_vectoring(x0: float, y0: float, m: int = 30, angles=None):
    """
    Vectoring mode: drives y → 0, accumulates total rotation angle.
      x_{i+1} = x_i + d_i*y_i*2^-i
      y_{i+1} = y_i - d_i*x_i*2^-i
      z_{i+1} = z_i + d_i*α_i
    Output: (x_m, y_m≈0, z≈atan(y0/x0))
    """
    if angles is None:
        angles = cordic_angles(m)
    x, y, z = x0, y0, 0.0
    for i in range(m):
        d = 1.0 if y >= 0.0 else -1.0
        x, y, z = x + d*y*(2.0**(-i)), y - d*x*(2.0**(-i)), z + d*angles[i]
    # scale factor same as rotation mode
    K = 1.0
    for i in range(m):
        K /= math.sqrt(1.0 + 2.0**(-2*i))
    return x, y, z, K



In [49]:
# ================================================================
# Q2 — Compute tan(θ)
# ================================================================
def cordic_tan(theta: float, m: int = 30, angles=None):
    """
    tan(θ) via rotation mode.
    Since (x,y) ≈ (1/K)(cosθ, sinθ), ratio y/x = tanθ (scale cancels).
    """
    a, ss, cc = reduce_to_first_quadrant(theta)
    x, y, K = cordic_rotate(a, m=m, angles=angles)
    t_first = y / x
    return (ss / cc) * t_first

## Q3

In [50]:
# ================================================================
# Q3 — Compute sin(θ) and cos(θ)
# ================================================================
def cordic_sin_cos(theta: float, m: int = 30, angles=None):
    """
    sin(θ), cos(θ) via rotation mode, corrected by K.
    (x,y) ≈ (1/K)(cosθ, sinθ)  ⇒ multiply by K to recover.
    """
    a, ss, cc = reduce_to_first_quadrant(theta)
    x, y, K = cordic_rotate(a, m=m, angles=angles)
    sin_a, cos_a = y * K, x * K
    return ss * sin_a, cc * cos_a


## Q4

In [51]:
# ================================================================
# Q4 — Compute atan(u) and arcsin(x)
# ================================================================
def cordic_atan(u: float, m: int = 30, angles=None):
    """arctan(u) via vectoring mode."""
    x, y, z, K = cordic_vectoring(1.0, u, m=m, angles=angles)
    return z

def cordic_arcsin(x: float, m: int = 30, angles=None):
    """
    arcsin(x) via identity:
        arcsin(x) = atan( x / sqrt(1 - x^2) )
    """
    if abs(x) > 1.0:
        return float("nan")
    if abs(1.0 - abs(x)) < 1e-18:
        return math.copysign(0.5*pi, x)
    denom = math.sqrt(max(0.0, 1.0 - x*x))
    return cordic_atan(x / denom, m=m, angles=angles)


## DEMOS

In [52]:
# ================================================================
# Demonstrations
# ================================================================
if __name__ == "__main__":
    ANG = cordic_angles(m=30)

    # Q2: tan demo
    print("=== Q2: tan via CORDIC ===")
    for th in [0.0, 0.1, 0.5, 1.0, -0.8, 1.2, 1.5]:
        t = cordic_tan(th, m=30, angles=ANG)
        print(f"θ={th: .3f}  tan≈{t: .12e}  math.tan={math.tan(th): .12e}  abs.err={abs(t-math.tan(th)): .2e}")

    # Q3: sin/cos demo
    print("\n=== Q3: sin & cos via CORDIC ===")
    for x in [0.1*k for k in range(0, 17)]:
        s, c = cordic_sin_cos(x, m=30, angles=ANG)
        print(f"x={x: .2f}  sin≈{s: .12e}  cos≈{c: .12e}  |Δsin|={abs(s-math.sin(x)): .2e}  |Δcos|={abs(c-math.cos(x)): .2e}")

    # Q4: atan demo
    print("\n=== Q4: atan via CORDIC ===")
    for u in [0.0, 0.1, 0.5, 1.0, 2.0, -1.5]:
        a = cordic_atan(u, m=30, angles=ANG)
        print(f"u={u: .2f}  atan≈{a: .12e}  math.atan={math.atan(u): .12e}  abs.err={abs(a-math.atan(u)): .2e}")

    # Q4: arcsin demo
    print("\n=== Q4: arcsin via CORDIC ===")
    for x in [-1.0, -0.5, 0.0, 0.3, 0.9, 1.0]:
        a = cordic_arcsin(x, m=30, angles=ANG)
        print(f"x={x: .2f}  arcsin≈{a: .12e}  math.asin={math.asin(x): .12e}  abs.err={abs(a-math.asin(x)): .2e}")

    # π via 4*atan(1)
    pi_est = 4.0 * cordic_atan(1.0, m=25, angles=ANG)
    print(f"\nπ via 4*atan(1): {pi_est:.12f} (error={abs(pi_est-math.pi):.2e})")


=== Q2: tan via CORDIC ===
θ= 0.000  tan≈ 1.526560380569e-09  math.tan= 0.000000000000e+00  abs.err= 1.53e-09
θ= 0.100  tan≈ 1.003346705731e-01  math.tan= 1.003346720855e-01  abs.err= 1.51e-09
θ= 0.500  tan≈ 5.463024891284e-01  math.tan= 5.463024898438e-01  abs.err= 7.15e-10
θ= 1.000  tan≈ 1.557407726105e+00  math.tan= 1.557407724655e+00  abs.err= 1.45e-09
θ=-0.800  tan≈-1.029638557192e+00  math.tan=-1.029638557050e+00  abs.err= 1.41e-10
θ= 1.200  tan≈ 2.572151630956e+00  math.tan= 2.572151622126e+00  abs.err= 8.83e-09
θ= 1.500  tan≈ 1.410141970662e+01  math.tan= 1.410141994717e+01  abs.err= 2.41e-07

=== Q3: sin & cos via CORDIC ===
x= 0.00  sin≈ 1.526560380569e-09  cos≈ 1.000000000000e+00  |Δsin|= 1.53e-09  |Δcos|= 3.33e-16
x= 0.10  sin≈ 9.983341515707e-02  cos≈ 9.950041654275e-01  |Δsin|= 1.49e-09  |Δcos|= 1.49e-10
x= 0.20  sin≈ 1.986693300402e-01  cos≈ 9.800665779943e-01  |Δsin|= 7.55e-10  |Δcos|= 1.53e-10
x= 0.30  sin≈ 2.955202059031e-01  cos≈ 9.553364893602e-01  |Δsin|= 7.58e-10 